In [11]:
import os

In [12]:
# turns the data in to a list + sorts it
dirs = sorted(os.listdir("data"))

In [13]:
# prints the subfolders in data 
dirs

['a1',
 'a2',
 'a3',
 'a4',
 'a5',
 'b1',
 'b2',
 'b3',
 'b4',
 'b5',
 'b6',
 'b7',
 'b8',
 'c1']

In [14]:
# for every subfolder
for directory in dirs:
    # get the individual filenames
    filenames = sorted(os.listdir(os.path.join("data", directory)))

    # for each individual file
    for text_file in filenames:
        print(os.path.join("data", directory, text_file))

        # load the data
        # process with spacy
        # save results

data/a1/0100.a1.txt
data/a1/0101.a1.txt
data/a1/0102.a1.txt
data/a1/0103.a1.txt
data/a1/0104.a1.txt
data/a1/0105.a1.txt
data/a1/0106.a1.txt
data/a1/0107.a1.txt
data/a1/0108.a1.txt
data/a1/0109.a1.txt
data/a1/0110.a1.txt
data/a1/0111.a1.txt
data/a1/0112.a1.txt
data/a1/0113.a1.txt
data/a1/0114.a1.txt
data/a1/0115.a1.txt
data/a1/0116.a1.txt
data/a1/0118.a1.txt
data/a1/0119.a1.txt
data/a1/0120.a1.txt
data/a1/0121.a1.txt
data/a1/0122.a1.txt
data/a1/0123.a1.txt
data/a1/0124.a1.txt
data/a1/0125.a1.txt
data/a1/0127.a1.txt
data/a1/0128.a1.txt
data/a1/0130.a1.txt
data/a1/0131.a1.txt
data/a1/0133.a1.txt
data/a1/0134.a1.txt
data/a1/0135.a1.txt
data/a1/0136.a1.txt
data/a1/0137.a1.txt
data/a1/0138.a1.txt
data/a1/0139.a1.txt
data/a1/0140.a1.txt
data/a1/0142.a1.txt
data/a1/0143.a1.txt
data/a1/0144.a1.txt
data/a1/0145.a1.txt
data/a1/0146.a1.txt
data/a1/0148.a1.txt
data/a1/0149.a1.txt
data/a1/0151.a1.txt
data/a1/0152.a1.txt
data/a1/0155.a1.txt
data/a1/0156.a1.txt
data/a1/0157.a1.txt
data/a1/0158.a1.txt
